In [108]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

In [143]:
pd.set_option('display.max_colwidth', None)

In [8]:
options = Options() 
driver = webdriver.Chrome(options=options)
driver.get('https://www.clasificadosonline.com/RealEstate.asp')

In [16]:
multi_check_box_toggle = driver.find_element(By.XPATH,'//*[@id="form1"]/div/table[2]/tbody/tr/td/div[1]')

In [12]:
select_all = driver.find_element(By.XPATH,'//*[@id="form1"]/div/table[2]/tbody/tr/td/div[2]/div[1]/input')

In [21]:
ver_listado = driver.find_element(By.XPATH,'//*[@id="BtnSearchListing"]')

In [23]:
multi_check_box_toggle.click()
time.sleep(2)
select_all.click()
time.sleep(2)
ver_listado.click()

In [29]:
order_by = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]')

In [30]:
ultimos_publicados = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]/option[7]')

In [31]:
order_by.click()

In [32]:
ultimos_publicados.click()

In [34]:
broker = driver.find_element(By.XPATH,'//*[@id="listing"]/table/tbody/tr/td/table/tbody/tr[2]/td/div/div[2]/table[1]/tbody/tr[1]/td/div[2]/table/tbody/tr/td[2]/table/tbody/tr[2]/td[2]/center/a/img')

In [40]:
broker.get_attribute('alt')

'ClasificadosOnline Costa Azul de Guardiola Realty    Lic. 16465'

In [42]:
driver.find_element(By.CSS_SELECTOR,'#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(1) > tbody > tr:nth-child(1) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span').text

'2 UNITS BEACH HOUSE  COSTA AZUL BEACH...'

In [ ]:
#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(1) > tbody > tr:nth-child(2) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span

In [ ]:
#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(1) > tbody > tr:nth-child(1) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span

In [ ]:
#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(3) > tbody > tr:nth-child(15) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span

In [46]:
range1 = range(1,4,2)
range2 = range(1,16)

for i, j in zip(range1, range2):
    print(i, j)

1 1
3 2


In [141]:
table_1_listings = []
table_2_listings = []
list_cards = []

for i in range(1,16):
    table_1 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(1)'
    table_1_title = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
    table_1_rooms = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
    table_1_price = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
    table_1_type = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
    table_1_location = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
    table_1_link = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'
    dictionary_cards = {}
    try:
        table_1_type_2 = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(5)'
        table_1_type_2_text = driver.find_element(By.CSS_SELECTOR, table_1_type_2).text
        dictionary_cards['Type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text + ' ' + table_1_type_2_text
        
    except NoSuchElementException:
        dictionary_cards['Type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text
    
    try:
        table_1_broker = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(2) > center > a > img'
        dictionary_cards['Broker'] = driver.find_element(By.CSS_SELECTOR, table_1_broker).get_attribute('alt')
    except NoSuchElementException:
        dictionary_cards['Broker'] = ''

    dictionary_cards['Title'] = driver.find_element(By.CSS_SELECTOR, table_1_title).text
    dictionary_cards['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_1_rooms).text
    dictionary_cards['Price'] = driver.find_element(By.CSS_SELECTOR, table_1_price).text
    dictionary_cards['Location'] = driver.find_element(By.CSS_SELECTOR, table_1_location).text
    dictionary_cards['Link'] = driver.find_element(By.CSS_SELECTOR, table_1_link).get_attribute('href')
    list_cards.append(dictionary_cards)

    table_2 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(3)'
    table_2_title = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
    table_1_listings.append(driver.find_element(By.CSS_SELECTOR,table_1_title).text)
    table_2_listings.append(driver.find_element(By.CSS_SELECTOR,table_2_title).text)


In [144]:
list_cards
df = pd.json_normalize(list_cards)
df

,Type,Broker,Title,Rooms,Price,Location,Link
0,", Casa",ClasificadosOnline Costa Azul de Guardiola Realty Lic. 16465,2 UNITS BEACH HOUSE COSTA AZUL BEACH...,5 Cuartos | 3 Baños,"$549,000","Urbanizacion-Costa Azul , Luquillo",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4742998
1,", Casa",ClasificadosOnline Villa Carolina de Team Realty,Excelente Propiedad ¡TERRERA de 2 uni...,4 Cuartos | 2 Baños,"$220,000","Urbanizacion-Villa Carolina , Carolina",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4730422
2,", Comercial Repo",ClasificadosOnline Cuyon de Nazza Realty,"Bo. Cuyón, Excelente oportunidad Inve...",5 Cuartos | 3 Baños,"$950,000","Barrio-Cuyon , Aibonito",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4708988
3,", Apartamento",ClasificadosOnline New San Juan de RICARDO REALTY,Compramos tú Casa AHORA! 787-613-6769...,2 Cuartos | 2 Baños,"$349,000","Condominio-New San Juan , Carolina",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4783500
4,", Casa",ClasificadosOnline Quebrada Seca de TUBELLAKASA.COM,EN PROCESO DE REMODELACIÓN,5 Cuartos | 3 Baños,"$249,000","Barrio-Quebrada Seca , Ceiba",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4716889
5,", Casa",ClasificadosOnline Villa Carolina de AA Real Estate,Vive de rentas Propiedad de 3 apartam...,>=6 Cuartos | 3 Baños,"$220,000","Urbanizacion-Villa Carolina , Carolina",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4770429
6,", Apartamento","ClasificadosOnline Jardines de Valencia de PREMIUM ASSETS SOLUTIONS, LLC",*opcionado*Cond. Jardines de Valencia...,2 Cuartos | 1 Baños,"$89,000","Condominio-Jardines de Valencia , San Juan - Hato Rey",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4707516
7,", Casa","ClasificadosOnline Marines de 1313 GROUP, LLC",URB MARINES REMODELADA,3 Cuartos | 1 Baños,"$120,000","Barrio-Marines , Fajardo",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4791173
8,", MultiFamiliar",ClasificadosOnline Jardines Metropolitanos de TU CASA PR,Urb Jardines Metropilitano! 8 apartam...,>=6 Cuartos | 5 Baños,"$390,000","Urbanizacion-Jardines Metropolitanos , San Juan - Hato Rey",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4724959
9,", Casa",ClasificadosOnline Venezuela de LINETTE MIRO REAL ESTATE,Bo.Venezuela DOS UNIDADES(contadores ...,5 Cuartos | 2 Baños,"$80,000","Barrio-Venezuela , San Juan - Río Piedras",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4665659


In [103]:
df = pd.json_normalize(list_cards)

In [104]:
df

,Title,Rooms,Price
0,2 UNITS BEACH HOUSE COSTA AZUL BEACH...,5 Cuartos | 3 Baños,"$549,000"
1,Excelente Propiedad ¡TERRERA de 2 uni...,4 Cuartos | 2 Baños,"$220,000"
2,"Bo. Cuyón, Excelente oportunidad Inve...",5 Cuartos | 3 Baños,"$950,000"
3,Compramos tú Casa AHORA! 787-613-6769...,2 Cuartos | 2 Baños,"$349,000"
4,EN PROCESO DE REMODELACIÓN,5 Cuartos | 3 Baños,"$249,000"
5,Vive de rentas Propiedad de 3 apartam...,>=6 Cuartos | 3 Baños,"$220,000"
6,*opcionado*Cond. Jardines de Valencia...,2 Cuartos | 1 Baños,"$89,000"
7,URB MARINES REMODELADA,3 Cuartos | 1 Baños,"$120,000"
8,Urb Jardines Metropilitano! 8 apartam...,>=6 Cuartos | 5 Baños,"$390,000"
9,Bo.Venezuela DOS UNIDADES(contadores ...,5 Cuartos | 2 Baños,"$80,000"


In [70]:
table_1_listings

['2 UNITS BEACH HOUSE  COSTA AZUL BEACH...',
 'Excelente Propiedad ¡TERRERA de 2 uni...',
 'Bo. Cuyón, Excelente oportunidad Inve...',
 'Compramos tú Casa AHORA! 787-613-6769...',
 'EN PROCESO DE REMODELACIÓN ',
 'Vive de rentas Propiedad de 3 apartam...',
 '*opcionado*Cond. Jardines de Valencia...',
 'URB MARINES REMODELADA',
 'Urb Jardines Metropilitano! 8 apartam...',
 'Bo.Venezuela DOS UNIDADES(contadores ...',
 'LOCAL COMERCIAL Y RESIDENCIA SEGUNDO ...',
 'Bo. Cedro Abajo',
 'Casa frente al mar (remodelada)',
 'PISO # 2....FRENTE AL MAR!!!!',
 'Local Comercial Toa Alta ! ']

In [71]:
table_2_listings

['** 15 Cuerdas en Barrio La Gloria **',
 'The Terrace',
 'MIRAMAR,8 APTS.SELLER FINANCING',
 'Venta Terreno Haciendas del Dorado',
 'Caldas Terrera- super lot.',
 'CASA GRANDE Y COMODA LLAME 787-817-95...',
 'COLINAS DE BAYAMON 9-A, GARDEN APT',
 'Income Property / Villa Carolina OPCI...',
 'Casa amueblada, 2100 ft, 3 cuartos, 2...',
 'Bo. Llanos',
 'CONDOMINIO BAHIA A SAN JUAN ',
 'MULTIFAMILIAR ¡Gran OPORTUNIDAD de in...',
 'Inversionista Oportunidad de Desarrol...',
 'Compramos tú Casa AHORA! 787-623-6769...',
 'EN REMODELACIÓN 3/2']